In [1]:
import csv
import json
import time
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def scrape_amazon_product(url):
    try:
        response = requests.get(url)
        if response.status_code == 404:
            print(f"{url} not available")
            return None

        soup = BeautifulSoup(response.content, 'html.parser')
        product_title = soup.find('span', {'id': 'productTitle'}).text.strip()
        product_image_url = soup.find('img', {'id': 'landingImage'})['src']
        product_price = soup.find('span', {'class': 'a-offscreen'}).text.strip()
        product_details = soup.find('div', {'id': 'productDescription'}).text.strip()

        product = {
            'product_title': product_title,
            'product_image_url': product_image_url,
            'product_price': product_price,
            'product_details': product_details
        }
        return product
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def scrape_amazon_products(csv_file):
    start_time = time.time()
    products = []

    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row

        for row in reader:
            country_code, asin = row
            url = f"https://www.amazon.{country_code}/dp/{asin}"
            product = scrape_amazon_product(url)
            if product:
                products.append(product)

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total time taken: {total_time} seconds")

    return products

csv_file = r"C:\Users\TY735LS\Downloads\urls.csv"
products = scrape_amazon_products(csv_file)

json_data = json.dumps(products, indent=4)
with open('output.json', 'w') as file:
    file.write(json_data)


Error scraping https://www.amazon.cn/dp/B0BSLK378P: 'NoneType' object has no attribute 'text'
Error scraping https://www.amazon.en/dp/4299043650: HTTPSConnectionPool(host='www.amazon.en', port=443): Max retries exceeded with url: /dp/4299043650 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000247E871F2B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Total time taken: 9.134693384170532 seconds


In [2]:
json_data

'[\n    {\n        "product_title": "Khooti Tablet Sleeve 7-inch D3 Jute Dark Grey",\n        "product_image_url": "https://m.media-amazon.com/images/I/51bl9P64UCL._SY300_SX300_QL70_ML2_.jpg",\n        "product_price": "\\u20b9299.00",\n        "product_details": "Large Capacity:The compartment can accommodate the power cable, adapter etc.New High Quality Material:Using lightweight, Supreme Quality FabricProtection for your Tablet:Comfortable & Softest Protection to your Tablet.Designed for Tablets:It can accommodate upto 7\\" Tablets.Custom Fits:Compatible with all 7\\" Tablets of all brands.AirPort Security Friendly:Increase the speed and efficiency of airport security by using Khooti Tablet Sleeves."\n    }\n]'

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def bypass_amazon_captcha(url, captcha_solution):
    options = Options()
    options.add_argument("--headless")  # Run Chrome in headless mode

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Find and solve the captcha
    captcha_input = driver.find_element(By.ID, 'captchacharacters')
    captcha_input.send_keys(captcha_solution)

    submit_button = driver.find_element(By.CSS_SELECTOR, 'input.a-button-input')
    submit_button.click()

    # Wait for the page to load after submitting the captcha
    time.sleep(3)

    # Perform further actions on the page if needed

    # Close the browser
    driver.quit()

url = 'https://www.amazon.com/errors/validateCaptcha'
captcha_solution = 'Your captcha solution'

bypass_amazon_captcha(url, captcha_solution)
